In [15]:
import pandas as pd
import math
import numpy as np

In [16]:
project_name = pd.read_csv("../DataFile/kickstarter.csv", usecols=["project_id", "pledged", "deadline", "launched_at", "backers_count", "percent_funded","project_category"])
project_details = pd.read_csv("../DataFile/kickstarter_details.csv", usecols=["project_id", "user_Facebook", "user_TotalProject"])
twitter_sentiment = pd.read_csv("../DataFile/twitter_sentiment.csv")
youtube_sentiment = pd.read_csv("../DataFile/youtube_sentiment.csv")


In [17]:
kickStarter = pd.merge(project_name, project_details, on='project_id',how='inner')
kickStarter = pd.merge(kickStarter, twitter_sentiment, left_on='project_id', right_on = 'Project_ID', how='inner')
kickStarter = pd.merge(kickStarter, youtube_sentiment, left_on='project_id', right_on = 'Product_id', how='left')

In [18]:
days = []
success = []
goal = []
facebook = []
has_Tweets = []
days_remaining = []
has_Youtube = []
youtube_score = []

In [19]:
kickStarter

,project_id,pledged,deadline,launched_at,backers_count,percent_funded,project_category,user_Facebook,user_TotalProject,Project_ID,numerator,Followers,LenTweets,tweet_count,Sentiment_Result,Product_id,sent_score
0,982188492,2170.0,1575088602,1573879002,14,217.000000,Painting,True,3,982188492,0.000000,0,0,0,0.000000,9.821885e+08,0.800000
1,1708879781,37735.0,1575089210,1573274810,186,134.767857,Farms,False,2,1708879781,0.000000,0,0,0,0.000000,NaN,NaN
2,1285560032,1741.0,1575089940,1572973216,50,174.100000,Tabletop Games,False,4,1285560032,0.000000,0,0,0,0.000000,1.285560e+09,-0.200000
3,689412203,18778.0,1575090000,1572494410,386,207.905226,Tabletop Games,False,5,689412203,205.000006,1942,2219,10,0.105561,6.894122e+08,0.133333
4,1461868634,9177.0,1575091800,1571946749,90,118.001800,Documentary,False,1,1461868634,0.000000,0,0,0,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,1677180482,981.0,1576413915,1573821915,25,49.050000,Comedy,True,7,1677180482,0.000000,0,0,0,0.000000,NaN,NaN
1075,1008845023,341.0,1576417110,1573825110,20,5.683333,Tabletop Games,False,1,1008845023,0.000000,0,0,0,0.000000,NaN,NaN
1076,551285503,1303.0,1576418413,1573826413,16,52.120000,Pop,True,1,551285503,0.000000,0,0,0,0.000000,NaN,NaN
1077,178129466,26.0,1576421341,1571233741,2,0.260000,Performance Art,False,1,178129466,0.000000,0,0,0,0.000000,NaN,NaN


In [20]:
for index, row in kickStarter.iterrows():
    days.append(int((row['deadline'] - row['launched_at']) / 86400))
    days_remaining.append(int((row['deadline'] - 1575079200) / 86400))
    
    if row['percent_funded'] > 90:
        success.append(1)
    else:
        success.append(0)

    goal.append((row['pledged'] * 100) / row['percent_funded'])
    
    if row['user_Facebook'] == True:
        facebook.append(1)
    else:
        facebook.append(0)
    
    if row['LenTweets'] > 1:
        has_Tweets.append(1)
    else:
        has_Tweets.append(0)
        
    if math.isnan(row['Product_id']):
        has_Youtube.append(0)
        youtube_score.append(0)
    else:
        has_Youtube.append(1)
        youtube_score.append(row['sent_score'])
    

In [21]:
kickStarter['days'] = days
kickStarter['success'] = success
kickStarter['goal'] = goal
kickStarter['facebook'] = facebook
kickStarter['hasTweets'] = has_Tweets
kickStarter['daysRemaining'] = days_remaining
kickStarter['has_Youtube'] = has_Youtube
kickStarter['youtube_sentiment'] = youtube_score

In [24]:
kickStarter[["project_id", "pledged", "backers_count", "percent_funded", "Followers",  "project_category", "user_TotalProject", "Sentiment_Result", "days", "success", "goal", "facebook", "hasTweets", "daysRemaining", "has_Youtube", "youtube_sentiment"]].to_csv('Modal_data.csv', mode = 'w' ,index=False)





In [10]:
kickStarter['hasTweets'].value_counts().to_dict()

{0: 683, 1: 396}

In [11]:
hasTweets = kickStarter[kickStarter['hasTweets'] == 1]

In [13]:
hasTweets.describe()

,project_id,pledged,deadline,launched_at,backers_count,percent_funded,user_TotalProject,Project_ID,numerator,Followers,...,Product_id,sent_score,days,success,goal,facebook,hasTweets,daysRemaining,has_Youtube,youtube_sentiment
count,3.960000e+02,3.960000e+02,3.960000e+02,3.960000e+02,396.000000,396.000000,396.000000,3.960000e+02,396.000000,3.960000e+02,...,2.600000e+01,26.000000,396.000000,396.000000,396.000000,396.000000,396.0,396.000000,396.000000,396.000000
mean,9.961126e+08,3.910081e+04,1.576117e+09,1.573393e+09,351.772727,399.148916,3.661616,9.961126e+08,15551.516135,6.374377e+04,...,1.210061e+09,0.369902,31.313131,0.669192,11265.328283,0.338384,1.0,11.515152,0.065657,0.024287
std,5.959767e+08,1.766492e+05,8.266100e+05,9.750094e+05,1477.952196,1433.222127,5.740078,5.959767e+08,69583.150059,2.198590e+05,...,5.253767e+08,0.262220,11.143389,0.471099,29419.045732,0.473758,0.0,9.567470,0.247994,0.112991
min,4.192230e+05,1.000000e+00,1.575087e+09,1.570054e+09,1.000000,0.000200,1.000000,4.192230e+05,-47646.200710,0.000000e+00,...,4.279018e+08,-0.135000,7.000000,0.000000,10.000000,0.000000,1.0,0.000000,0.000000,-0.135000
25%,5.202231e+08,1.279173e+03,1.575505e+09,1.572881e+09,29.750000,52.269457,1.000000,5.202231e+08,27.325000,5.855000e+02,...,7.046267e+08,0.224167,28.000000,0.000000,1200.000000,0.000000,1.0,4.000000,0.000000,0.000000
50%,9.522650e+08,4.507000e+03,1.576001e+09,1.573546e+09,78.000000,124.370000,1.500000,9.522650e+08,385.650012,2.843000e+03,...,1.249114e+09,0.395000,30.000000,1.000000,4735.000000,0.000000,1.0,10.000000,0.000000,0.000000
75%,1.465742e+09,1.467150e+04,1.576504e+09,1.574176e+09,206.500000,268.535390,4.000000,1.465742e+09,4218.150056,4.902400e+04,...,1.612926e+09,0.547727,32.250000,1.000000,10675.000000,1.000000,1.0,16.000000,0.000000,0.000000
max,2.143160e+09,2.186420e+06,1.579447e+09,1.575042e+09,20771.000000,23030.000000,55.000000,2.143160e+09,798623.515174,2.450455e+06,...,2.143160e+09,0.900000,60.000000,1.000000,500000.000000,1.000000,1.0,50.000000,1.000000,0.900000


In [14]:
noTweets = kickStarter[kickStarter['hasTweets'] == 0]
noTweets.describe()

,project_id,pledged,deadline,launched_at,backers_count,percent_funded,user_TotalProject,Project_ID,numerator,Followers,...,Product_id,sent_score,days,success,goal,facebook,hasTweets,daysRemaining,has_Youtube,youtube_sentiment
count,6.830000e+02,683.000000,6.830000e+02,6.830000e+02,683.000000,683.000000,683.000000,6.830000e+02,683.0,683.0,...,3.600000e+01,36.000000,683.000000,683.000000,6.830000e+02,683.000000,683.0,683.000000,683.000000,683.000000
mean,1.083302e+09,4188.014773,1.575810e+09,1.572602e+09,41.259151,146.936049,1.903367,1.083302e+09,0.0,0.0,...,1.212925e+09,0.249481,36.972182,0.323572,2.340833e+04,0.276720,0.0,7.934114,0.052709,0.013150
std,6.313499e+08,12595.053612,6.025840e+05,1.252603e+06,85.354283,866.375214,3.199529,6.313499e+08,0.0,0.0,...,6.777325e+08,0.243146,13.689501,0.468182,1.287074e+05,0.447705,0.0,6.961355,0.223615,0.078398
min,5.735958e+06,1.000000,1.575083e+09,1.569333e+09,1.000000,0.000040,1.000000,5.735958e+06,0.0,0.0,...,1.047783e+08,-0.200000,9.000000,0.000000,1.000000e+00,0.000000,0.0,0.000000,0.000000,-0.200000
25%,5.183754e+08,46.000000,1.575314e+09,1.571788e+09,3.000000,1.000500,1.000000,5.183754e+08,0.0,0.0,...,5.112198e+08,0.088889,30.000000,0.000000,1.350000e+03,0.000000,0.0,2.000000,0.000000,0.000000
50%,1.097215e+09,529.000000,1.575741e+09,1.572912e+09,12.000000,16.373333,1.000000,1.097215e+09,0.0,0.0,...,1.371991e+09,0.263333,30.000000,0.000000,5.000000e+03,0.000000,0.0,7.000000,0.000000,0.000000
75%,1.640130e+09,2901.500000,1.576173e+09,1.573524e+09,39.000000,110.442735,2.000000,1.640130e+09,0.0,0.0,...,1.797054e+09,0.349790,45.000000,1.000000,1.142500e+04,1.000000,0.0,12.000000,0.000000,0.000000
max,2.145129e+09,215506.000000,1.579272e+09,1.575066e+09,1073.000000,16933.600000,41.000000,2.145129e+09,0.0,0.0,...,2.125004e+09,0.800000,68.000000,1.000000,2.500000e+06,1.000000,0.0,48.000000,1.000000,0.800000
